In [3]:
import os
from glob import glob
from tqdm import tqdm
import tensorflow.compat.v1 as tf

tf.enable_eager_execution()

from waymo_open_dataset import dataset_pb2 as open_dataset
from waymo_open_dataset import label_pb2

In [33]:
records_dir = '/data/dataset/waymo/train/'

IMG_WIDTH = 1920
IMG_HEIGHT = 1280
# directions = {
#     'FRONT' : open_dataset.CameraName.FRONT, 
#     'FRONT_LEFT' : open_dataset.CameraName.FRONT_LEFT, 
#     'FRONT_RIGHT' : open_dataset.CameraName.FRONT_RIGHT,
#     # 'SIDE_LEFT' : open_dataset.CameraName.SIDE_LEFT,
#     # 'SIDE_RIGHT' : open_dataset.CameraName.SIDE_RIGHT
#     }
directions = [open_dataset.CameraName.FRONT_LEFT, open_dataset.CameraName.FRONT, open_dataset.CameraName.FRONT_RIGHT]

def cnt_ped(frame):
    for cam_obj in frame.context.stats.camera_object_counts:
        # print(cam_obj)
        if cam_obj.type == label_pb2.Label.Type.TYPE_PEDESTRIAN:
            return cam_obj.count
    return 0

def get_bbox_loc(box, camera_direction, pano: bool = False) -> list:
    # print(type(box))
    _bbox_begin_x = box.center_x - 0.5 * box.length
    _bbox_begin_y = box.center_y - 0.5 * box.width
    _bbox_end_x = _bbox_begin_x + box.length
    _bbox_end_y = _bbox_begin_y + box.width
    if pano:
        if camera_direction == open_dataset.CameraName.FRONT:
            _bbox_begin_x += IMG_WIDTH
            _bbox_end_x += IMG_WIDTH
        elif camera_direction == open_dataset.CameraName.FRONT_RIGHT:
            _bbox_begin_x += IMG_WIDTH * 2
            _bbox_end_x += IMG_WIDTH * 2
    return [_bbox_begin_x, _bbox_begin_y, _bbox_end_x, _bbox_end_y]    

tfrecords = glob(records_dir + '*.tfrecord')
tfrecord: str
for tfrecord in tqdm(tfrecords, "tfrecord : "):
    _dataset = tf.data.TFRecordDataset(tfrecord, compression_type='')
    _record_name = os.path.basename(tfrecord)
    _cnt_frame = 0
    for data in tqdm(_dataset, "dataset : "): 
        if _cnt_frame % 50 != 0:
            # 50frame(=5s)ごとに書き出し
            _cnt_frame += 1
            continue
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        if cnt_ped(frame) == 0:
            continue
        for camera_labels in frame.camera_labels:
            # print(type(camera_labels))
            camera_direction = camera_labels.name
            if not camera_direction in directions:
                continue
            # print(camera_direction)
            for ann_label in camera_labels.labels:
                print(type(ann_label.box))
                # print(type(camera_direction))
                print(ann_label.id)
                print(get_bbox_loc(ann_label.box, camera_direction))
                # print(type(ann_label.type))
                print("↑", label_pb2.Label.Type.Name(ann_label.type), end='\n\n')
                break
            #     print("anntype",ann_label.type, end='')
        frame_name = frame.context.name + "-" + format(_cnt_frame, '03d')
        _cnt_frame += 1
    #     break
    # break

dataset : : 198it [00:01, 189.48it/s]:00<?, ?it/s]
tfrecord :   0%|          | 1/798 [00:01<14:06,  1.06s/it]

<class 'waymo_open_dataset.label_pb2.Box'>
02f3a769-021f-49b0-b954-18e4fa1d5fde
[893.06826, 654.3272400000001, 906.96324, 677.0644800000001]
↑ TYPE_VEHICLE

<class 'waymo_open_dataset.label_pb2.Box'>
0ec04ccf-c924-45b1-ab54-cbc82a742877
[0.0, 936.64797, 1407.8141099999998, 1280.0]
↑ TYPE_VEHICLE

<class 'waymo_open_dataset.label_pb2.Box'>
42d3fd29-cc31-44c6-a044-7af1a172488e
[292.741965, 707.0650050000002, 346.427115, 787.2769350000002]
↑ TYPE_PEDESTRIAN

<class 'waymo_open_dataset.label_pb2.Box'>
0611ea50-f652-406d-942a-9ec7e3e90546
[1369.91871, 655.59042, 1397.70867, 692.22264]
↑ TYPE_PEDESTRIAN

<class 'waymo_open_dataset.label_pb2.Box'>
0bc3d489-bdb2-47d2-811b-fd69abef6ffd
[366.006405, 574.7469, 457.586955, 672.01176]
↑ TYPE_VEHICLE

<class 'waymo_open_dataset.label_pb2.Box'>
08ac1a40-1f4d-42a5-b3a5-24ea616d58dc
[83.685675, 663.80109, 131.054925, 797.69817]
↑ TYPE_PEDESTRIAN

<class 'waymo_open_dataset.label_pb2.Box'>
02f3a769-021f-49b0-b954-18e4fa1d5fde
[911.6062805997583, 638.477

dataset : : 199it [00:00, 583.12it/s]
tfrecord :   0%|          | 2/798 [00:01<08:32,  1.55it/s]

<class 'waymo_open_dataset.label_pb2.Box'>
02f3a769-021f-49b0-b954-18e4fa1d5fde
[910.7527799999999, 638.0781573217491, 924.64776, 670.691201252244]
↑ TYPE_VEHICLE

<class 'waymo_open_dataset.label_pb2.Box'>
0bc3d489-bdb2-47d2-811b-fd69abef6ffd
[361.49505913542293, 576.1454116916616, 454.51922086457705, 676.5682216916616]
↑ TYPE_VEHICLE

<class 'waymo_open_dataset.label_pb2.Box'>
08ac1a40-1f4d-42a5-b3a5-24ea616d58dc
[72.00126, 660.9589349999999, 133.89708000000002, 803.0666849999999]
↑ TYPE_PEDESTRIAN



<class 'waymo_open_dataset.label_pb2.Box'>
1a7162c2-e63a-42b2-96dd-9f3ba7ee5568
[764.8554899999999, 682.432995, 854.5412699999999, 757.592205]
↑ TYPE_VEHICLE

<class 'waymo_open_dataset.label_pb2.Box'>
1a7162c2-e63a-42b2-96dd-9f3ba7ee5568
[745.90779, 682.1172, 840.64629, 757.908]
↑ TYPE_VEHICLE

<class 'waymo_open_dataset.label_pb2.Box'>
2ca4909d-eee7-43c7-bcab-65e76cbd86c6
[315.79499999999996, 704.8544400000001, 469.90295999999995, 1042.1235000000001]
↑ TYPE_PEDESTRIAN



...

dataset : : 198it [00:00, 397.40it/s]
tfrecord :   1%|          | 6/798 [00:03<06:13,  2.12it/s]

<class 'waymo_open_dataset.label_pb2.Box'>
30c250b3-342e-4611-afbc-6078a5c69451
[1185.8102250000002, 702.32808, 1205.389515, 741.48666]
↑ TYPE_VEHICLE

<class 'waymo_open_dataset.label_pb2.Box'>
8baace0f-97db-4ad7-a313-43517e51864d
[0.631590000000017, 714.9598799999999, 306.32115, 799.5929399999999]
↑ TYPE_VEHICLE

<class 'waymo_open_dataset.label_pb2.Box'>
71990a7e-5c37-4ffd-aca2-5ea323eb2726
[928.4373, 653.379855, 1399.6034399999999, 897.805185]
↑ TYPE_VEHICLE

<class 'waymo_open_dataset.label_pb2.Box'>
10b9089f-14fc-46cf-b427-72547cba7386
[1808.557965, 1111.2826049999999, 1920.0, 1280.0]
↑ TYPE_VEHICLE

<class 'waymo_open_dataset.label_pb2.Box'>
5002eb88-da43-4e02-b177-e0282c7df7cb
[1632.02856, 654.6430349999999, 1674.97668, 726.012705]
↑ TYPE_PEDESTRIAN

<class 'waymo_open_dataset.label_pb2.Box'>
8cd6fec6-46f0-4705-9644-a30f6a6af8e8
[90.94895999999994, 885.173385, 1920.0, 1280.0]
↑ TYPE_VEHICLE



dataset : : 155it [00:00, 391.66it/s]
tfrecord :   1%|          | 6/798 [00:03<07:58,  1.66it/s]


<class 'waymo_open_dataset.label_pb2.Box'>
12a3e32d-9dce-4490-8cdc-1c79834fb757
[0.0, 688.1193991431688, 475.63325165641737, 928.9679684080539]
↑ TYPE_VEHICLE

<class 'waymo_open_dataset.label_pb2.Box'>
0d62a7e9-f6b9-45b5-a1c5-1a99eb8353ca
[1883.0855850000003, 701.69649, 1915.2966750000003, 732.0128100000001]
↑ TYPE_PEDESTRIAN

<class 'waymo_open_dataset.label_pb2.Box'>
426e7d2f-ee35-4f74-a876-5fdb531af410
[0.0, 666.643245, 174.634635, 1043.702475]
↑ TYPE_VEHICLE



KeyboardInterrupt: 